In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url="https://taiko.namco-ch.net/taiko/en/songlist/"

In [2]:
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

music_categories = [cat.get('href') for cat in soup.select_one('#sgnavi').select('a')]

In [4]:
music_categories

['pops.php#sgnavi',
 'kids.php#sgnavi',
 'anime.php#sgnavi',
 'vocaloid.php#sgnavi',
 'game.php#sgnavi',
 'variety.php#sgnavi',
 'classic.php#sgnavi',
 'namco.php#sgnavi']

In [5]:
pops_res = requests.get(url + music_categories[0])
pops_soup = BeautifulSoup(pops_res.content, 'html.parser')
pops_table = pops_soup.select_one('table')
pops_rows = pops_table.select('tr')[2:]
pops_tds = pops_rows[0].select('td')

In [7]:
pops_rows[0]

<tr>
<th>
                ケセラセラ              <span class="new">NEW!</span>
<p></p>
</th>
<td></td>
<td>2</td>
<td>3</td>
<td>4</td>
<td>7</td>
<td>-</td>
</tr>

In [8]:
namco_res = requests.get(url + music_categories[-1])
namco_soup = BeautifulSoup(namco_res.content, 'html.parser')
namco_table = namco_soup.select_one('table')
namco_rows = namco_table.select('tr')[2:]
namco_rows[0]

<tr>
<th>
                沓の子を打つ              <span class="new">NEW!</span>
<p>風薙ぎ(kazenagh)</p>
</th>
<td></td>
<td>5</td>
<td>7</td>
<td>8</td>
<td>8</td>
<td>10</td>
<td class="youtube"><a href="https://www.youtube.com/watch?v=ohIqE6dBsfw" target="_blank"><img alt="動画" src="../images/songlist/btn_movie.png"/></a></td>
</tr>

In [9]:
song_titles = []
artists = []
categories = []
easy_lv = []
normal_lv = []
hard_lv = []
extreme_lv = []
inner_extreme_lv = []

for cat in music_categories:
    category_name = cat.split('.')[0]
    cat_res = requests.get(url + cat)
    category_soup = BeautifulSoup(cat_res.content, 'html.parser')
    rows = category_soup.select_one('table').select('tr')[2:]
    for row in rows:
        info = (row.select_one('th').get_text(strip=True, separator='|')).split('|')
        if len(info) > 1:
            title, *others, artist = info
        else:
            title = info[0]

        papamama, easy, normal, hard, extreme, inner_extreme, *_ = row.select('td')
        song_titles.append(title)
        artists.append(artist)
        categories.append(category_name)
        easy_lv.append(int(easy.text))
        normal_lv.append(int(normal.text))
        hard_lv.append(int(hard.text))
        extreme_lv.append(int(extreme.text))
        inner_extreme_lv.append(None if inner_extreme.text == '-' else int(inner_extreme.text))

In [10]:
df = pd.DataFrame({
    'Song Title': song_titles,
    'Category': categories,
    'Artist': artists,
    'Easy': easy_lv,
    'Normal': normal_lv,
    'Hard': hard_lv,
    'Extreme': extreme_lv,
    'Extreme (Inner)': inner_extreme_lv
    
})

df

,Song Title,Category,Artist,Easy,Normal,Hard,Extreme,Extreme (Inner)
0,ケセラセラ,pops,NEW!,2,3,4,7,NaN
1,ライラック,pops,TVアニメ「忘却バッテリー」より,3,4,4,8,NaN
2,NEW KAWAII,pops,FRUITS ZIPPER,3,5,5,8,NaN
3,わたしの一番かわいいところ,pops,FRUITS ZIPPER,2,3,5,7,NaN
4,イガク,pops,原口沙輔 feat. 重音テト,3,5,6,8,10.0
...,...,...,...,...,...,...,...,...
1358,スーハー2000,namco,steμ feat.siroa,5,7,8,10,NaN
1359,ドンカマ2000,namco,steμ feat.siroa,5,7,8,10,NaN
1360,ノるどん2000,namco,steμ feat.siroa,4,6,7,10,NaN
1361,まださいたま2000,namco,steμ feat.siroa,5,7,8,10,NaN
